In [20]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf

In [21]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = TFAutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

Some layers from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [22]:
tokens = tokenizer.encode("It was a terrible day.",return_tensors="tf")


In [23]:
outputs = model(tokens)
outputs

TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(1, 5), dtype=float32, numpy=
array([[ 2.9939466 ,  1.1758162 , -0.49353743, -2.0897737 , -1.4070722 ]],
      dtype=float32)>, hidden_states=None, attentions=None)

In [24]:
outputs.logits

<tf.Tensor: shape=(1, 5), dtype=float32, numpy=
array([[ 2.9939466 ,  1.1758162 , -0.49353743, -2.0897737 , -1.4070722 ]],
      dtype=float32)>

In [25]:
int(tf.argmax(outputs.logits,axis=1))+1
#Shows 1st which is 2.9939466

1

In [26]:
import requests
from bs4 import BeautifulSoup
import re

In [27]:
r = requests.get('https://www.yelp.com/biz/old-ottoman-cafe-ve-restaurant-istanbul')
soup = BeautifulSoup(r.text,"html.parser")
regex = re.compile(".*comment.*")
results = soup.find_all("p",{"class":regex})
reviews = [result.text for result in results]

In [28]:
reviews

['best food experience in Istanbul, our host, Ali was very pro and kind. Food quality was out of this world. Ambiance was so calm and beautiful.I wish I could give them much more stars than 5. Oh, forgot to mention Turkish white wine was amazing',
 'Good service. Good food. They have the clay pot dish. Really good Turkish rice. This is a place for tourists. Reasonable price for tourists.',
 'We went here for lunch, the restaurant pretty empty but warm which were perfect for us. We ordered a claypot beef for one person, together with rice and breads definitely enough to feed the two of us along with babaganoush and olives.Would recommend the place! Price was ££',
 'This restaurant was the first place a dined at upon arriving in Istanbul 3 weeks ago. I, unfortunately, cannot remember what we ordered, but, I can tell you we walked away stuffed and extremely satisfied with our meal. We ordered several items and shared it all. The wait staff was very friendly and we enjoyed the whole experi

In [29]:
import numpy as np
import pandas as pd

In [30]:
df = pd.DataFrame(np.array(reviews),columns=["review"])

In [31]:
df["review"].iloc[1]

'Good service. Good food. They have the clay pot dish. Really good Turkish rice. This is a place for tourists. Reasonable price for tourists.'

In [32]:
def score(review):
    tokens = tokenizer.encode(review,return_tensors="tf")
    outputs = model(tokens)
    outputs.logits
    return int(tf.argmax(outputs.logits,axis=1))+1

In [33]:
df['Score'] = df['review'].apply(lambda x: score(x[:512]))


In [34]:
len(df)

10

In [35]:
# In order to show to model's accurate adding mock data using GPT-3
mockdata = [
    'The food was amazing and the service was top-notch!',
    'The restaurant had great ambiance, but the food was just okay.',
    'The food was delicious, but the service needs improvement.',
    'The staff was friendly and attentive, making our experience enjoyable.',
    'The dishes were subpar and overpriced, not worth the visit.',
    'The restaurant had a cozy atmosphere, perfect for a date night.',
    'The food was outstanding, and the presentation was impressive.',
    'The service was slow and the staff seemed disorganized.',
    'The restaurant offers a wide variety of dishes to suit every palate.',
    'The food was fresh and flavorful, exceeding our expectations.',
    'The portions were small, leaving us unsatisfied.',
    'The waitstaff was knowledgeable and made excellent recommendations.',
    'The restaurant has a great view, perfect for special occasions.',
    'The food was undercooked and the flavors were bland.',
    'The prices were reasonable, considering the quality of the food.',
    'The service was prompt and efficient, ensuring a smooth dining experience.',
    'The restaurant was crowded, leading to longer wait times.',
    'The menu lacked vegetarian options, limiting choices for some guests.',
    'The desserts were delectable and the perfect ending to our meal.',
    'The staff went above and beyond to accommodate our dietary restrictions.',
    'The food was disappointing, and we left unsatisfied.',
    'The atmosphere was lively and energetic, adding to the dining experience.',
    'The restaurant had a unique and creative menu, offering something new.',
    'The service was friendly and attentive, making us feel welcomed.',
    'The flavors were well-balanced, and the dishes were beautifully presented.',
    'The restaurant had a warm and inviting ambiance, perfect for a relaxed meal.',
    'The food was mediocre, not living up to the hype.',
    'The portions were generous, leaving us feeling satisfied and content.',
    'The staff was unprofessional and inattentive, negatively impacting our visit.',
    'The restaurant had a diverse wine selection, complementing the meal perfectly.',
]
dfmock = pd.DataFrame(mockdata,columns=["review"])
dfmock['Score'] = dfmock['review'].apply(lambda x: score(x[:512]))


In [36]:
dfmock

,review,Score
0,The food was amazing and the service was top-n...,5
1,"The restaurant had great ambiance, but the foo...",3
2,"The food was delicious, but the service needs ...",3
3,"The staff was friendly and attentive, making o...",4
4,"The dishes were subpar and overpriced, not wor...",2
5,"The restaurant had a cozy atmosphere, perfect ...",4
6,"The food was outstanding, and the presentation...",5
7,The service was slow and the staff seemed diso...,2
8,The restaurant offers a wide variety of dishes...,5
9,"The food was fresh and flavorful, exceeding ou...",5
